## Задание 1

Свертка списка - это обобщенная операция над списком, c помощью которой можно преобразовать список в единое значение. Например, рассмотрим реализации свертки слева и свертки справа (левоассоциативную свертку и правоассоциативную свертку):

In [1]:
def foldl(f, x0, lst):
    if not lst:
        return x0
    return foldl(f, f(x0, lst[0]), lst[1:])


def foldr(f, x0, lst):
    if not lst:
        return x0
    return f(lst[0], foldr(f, x0, lst[1:]))

In [2]:
#сумма
foldl(lambda x, y: x + y, 0, [1, 2, 3])

6

In [3]:
#конкатенация
foldl(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

'123'

In [4]:
f = lambda x, y: x / y
foldl(f, 1, [1, 2, 3]), \
foldr(f, 1, [1, 2, 3])

(0.16666666666666666, 1.5)

Задача: реализовать foldl через foldr и наоборот. Вместо многоточий нужно вставить выражения, которые бы привели к нужному результату. Модифицировать сам список нельзя.

In [8]:
def foldl2(f, x0, lst):
    return foldr(lambda x, y: lambda n: y(f(n, x)), lambda m: m, lst)(x0)


def foldr2(f, x0, lst):
    return foldl(lambda y, x: lambda n: y(f(x, n)), lambda m: m, lst)(x0)

In [10]:
foldl2(lambda x, y: x + y, 32, [10, -4, 7])

45

In [11]:
foldl2(lambda x, y: '{}{}'.format(x, y), 'be the', [' light', ' for', ' people'])

'be the light for people'

In [12]:
f = lambda x, y: x / y
foldl2(f, 12, [5, 1, 2]), \
foldr2(f, 12, [5, 1, 2])

(1.2, 0.8333333333333334)

## Задание 2

нужно написать функцию, которая принимает две строки и проверяет, входит ли хотя бы одна перестановка второй строки в первую. Например:

a = 'abcrotm'

b = 'tro'

функция def check_inv(a, b) вернет True, так как 'rot' содержится в 'abcrotm'. Нужно подумать как можно более оптимальный алгоритм и оценить его сложность.

In [48]:
def check_inv(a, b):
    
    flag = False
    
    index = {}
    b_chars = [char for char in b]

    for i in range(len(a)):
        if a[i] in b:
            index[i] = a[i]

            
    may_be = []
    for key, value in index.items():
        A = list(range(key, key + len(b)))
        B = list(index.keys())
        if set(A).issubset(B):
            for k in A:
                may_be.append(index[k])
        break
        

    if sorted(may_be) == sorted(b_chars):
        flag = True
            
    return flag

In [51]:
a = 'mrmkmkrmkksswpwdcdmckmek'
b = 'mkkcme'
check_inv(a, b)

True

In [49]:
a = 'kkdkdkkkkkkdkkkkkeeeee'
b = 'ekkd'
check_inv(a, b)

False

In [50]:
check_inv('abb', 'aab')

False

## Задание 3

Реализовать бинарное дерево (класс Tree), в нём методы repr, str, iter (итерация только по листьям).

In [12]:
class Tree:
    
    def __init__(self, value=None, left=None, right=None):
        self.left = left
        self.right = right
        self.value = value
        
        
    def __iter__(self):
        if self.left == None and self.right == None:
            yield self.value
        
        if self.left:
            yield from self.left
        
        if self.right:
            yield from self.right
    
    
    def __str__(self):
        res = [str(self.value)]
        
        if self.left:
            res.append(self.left.__str__())
        
        if self.right:
            res.append(self.right.__str__())
            
        return ' '.join(res)
    
    
    def __repr__(self):
        if self.left == None and self.right == None:
            return f"Tree({self.value})"
        
        if self.right == None :
            return f"Tree({self.value}, {self.left.__repr__()})"
        
        if self.left == None:
            return f"Tree({self.value}, None, {self.right.__repr__()})"
        
        return f"Tree({self.value}, {self.left.__repr__()}, {self.right.__repr__()})"
    
    

tree = Tree(0, Tree(1, Tree(3), Tree(4)),                             
               Tree(2))

list(tree) == [3, 4, 2]

True

In [13]:
print(tree)

0 1 3 4 2


In [14]:
tree

Tree(0, Tree(1, Tree(3), Tree(4)), Tree(2))

## Задание 4

Реализовать простейший калькулятор математических выражений:

1) только целые числа \
2) +, -, *, \ \
3) скобки

Можно использовать регулярные выражения

In [39]:
operations = {'+': (0, lambda x, y: x + y),
              '-': (0, lambda x, y: x - y),
              '*': (1, lambda x, y: x * y),
              '/': (1, lambda x, y: x / y)}

In [54]:
def parse(string):
    
    number = ''
    result = []
    
    for symb in string:
        
        if symb in '1234567890.,': 
            number += symb  
            
        elif number: 
            result.append(float(number)) 
            number = ''
            
        if symb in '*/+-()': 
            result.append(symb)
            
    if number:  
        result.append(float(number))  
        
    return result

In [55]:
def writeback(parsed_string):
    
    stack = [] 
    result = []
    
    for token in parsed_string:
        
        if token in operations: 
            while stack and stack[-1] != "(" and operations[token][0] <= operations[stack[-1]][0]:
                result.append(stack.pop())
            stack.append(token)
            
        elif token == ")":
            while stack:
                x = stack.pop()
                if x == "(":
                    break
                
                result.append(x)
                
        elif token == "(":
            stack.append(token)
            
        else:
            result.append(token)
            
    while stack:
        result.append(stack.pop())
        
    return result

In [59]:
def calc(string):
    
    string = writeback(parse(string))
    stack = []
    
    for token in string:
        
        if token not in operations:
            stack.append(token)
            
        if token in operations:
            first, second = stack.pop(), stack.pop()
            stack.append(operations[token][1](second, first))
            
    return stack.pop()

In [60]:
string = '23 * 54 / 12 * (1 / 3.7)'
eval(string) == calc(string)

True

In [61]:
string = '((24 - 53 / 46 * (567 +3.678 / (173 * (7 - 845674)))))'
eval(string) == calc(string)

True